In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from subprocess import check_output
from sklearn import metrics, model_selection
# /Users/mli3/ml/Kaggle/train.csv

def load_data(csv_path):
    return pd.read_csv(csv_path)

# load the data
csv_path = '/Users/mli3/ml/Kaggle/'
train = load_data(csv_path + "train.csv")
test = load_data(csv_path + "test.csv")


train.describe()

# remove columns where the value is the same for all rows (no new information)
nunique = train.apply(pd.Series.nunique)
col_same_train = list(nunique[nunique==1].index)
train2 = train.drop(col_same_train,1)

# apply the same to the test set
test2 = test.drop(col_same_train, 1)
# remove columns where the column is same as another column

# hist of Y

import matplotlib.pyplot as plt
y_train = train2[["y"]]
y_train.hist(bins=200, figsize=(20,15))
#plt.show()

# remove row without the maximum y (looks like outliet)
outlier_limit = 160
train3 = train2.loc[train2['y']<=outlier_limit]


print("We removed all data y >" + str(outlier_limit))
print("There were " + str(sum(train2['y']>outlier_limit)) + " outliers")

c = train3.corr()
s = c.unstack()
so = s.sort_values(kind='quicksort', ascending=False)
so['y']

# remove columns that are identical and also those that are complete opposite
col_same = s[((s==1) | (s==-1)) & (s.index.get_level_values(0) != s.index.get_level_values(1))]
#keep the smaller of the columns
# if x is smaller than the other columns, throw other columns out, else ignore
dup_columns = set()
for x,y in col_same.index:
    if int(x[1:]) < int(y[1:]):
        dup_columns.add(y)

# there are 45+8 dup_columns
train4 = train3.drop(list(dup_columns), 1)
test4 = test2.drop(list(dup_columns), 1)
# impute missing values

# # this part might be unecessary
# from sklearn.preprocessing import Imputer

# imputer = Imputer(strategy="median")

# df_dedup_num = df_dedup.drop(["ID", "y", "X0",  "X1",  "X2", "X3", "X4", "X5", "X6", "X8"], axis=1)
# imputer.fit(df_dedup_num)
# imputer.statistics_
# df_dedup_num.median().values
# X = imputer.transform(df_dedup_num)
# df_dedup_num_tr = pd.DataFrame(X, columns=df_dedup_num.columns)

# 1 hot encoding


train_num = train4.drop(["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"], 1) # kept y column
train_cat = train4.loc[:, ["ID", "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]]



test_num = test4.drop(["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"], 1)
test_cat = test4.loc[:, ["ID", "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]]
#from sklearn.preprocessing import LabelBinarizer
#encoder = LabelBinarizer()
#data_dedup_cat_1hot = encoder.fit_transform(data_dedup_cat)
#data_dedup_cat_1hot


train_one_hot_cat = pd.get_dummies(train_cat, columns=["X0", "X1",  "X2", "X3", "X4", "X5", "X6", "X8"])
train_one_hot = train_one_hot_cat.merge(train_num, on='ID')

test_one_hot_cat = pd.get_dummies(test_cat, columns=["X0", "X1",  "X2", "X3", "X4", "X5", "X6", "X8"])
test_one_hot = test_one_hot_cat.merge(test_num, on='ID')

X = train_one_hot.drop(['ID', 'y'], axis=1)
y = train_one_hot['y']


feature_labels = X.columns
forest = RandomForestRegressor(n_estimators=1000, random_state=0, n_jobs=-1)
forest.fit(X, y)    # Fits the Random Forest Regressor to the entire data set.
importances = forest.feature_importances_  # Sets importances equal to the feature importances of the model
indices = np.argsort(importances)[::-1]
order_features = []
order_importances = []
for f in range(X.shape[1]):
    #print("%2d) %-*s %f" % (f+1, 30, feature_labels[indices[f]], importances[indices[f]]))
    order_features.append(feature_labels[f])
    order_importances.append(importances[indices[f]])





# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score




We removed all data y >160
There were 5 outliers


In [8]:

ncol=350
print("Training is done using " + str(ncol) + " columns")
train = train_one_hot.drop(order_features[ncol:], axis=1) # Modify train to only take in the top 100 features and the target column y
test = test_one_hot.loc[:,list(train.columns)]
test = test.drop(["y"], axis=1).fillna(0)

y_train = train['y']
train = train.drop(["y"], axis=1)


#print("Using PCA")
#from sklearn import decomposition
#pca = decomposition.PCA()
#pca.fit(X)

#train = pca.fit_transform(train)[:,:ncol]
#test = pca.transform(test)[:,:ncol]




from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("    Scores:", scores)
    print("    Mean:", scores.mean())
    print("    Std dev:", scores.std())


# Linear Regression
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(train, y_train)
lin_reg_scores = cross_val_score(lin_reg, train, y_train, 
    scoring="r2", cv=10)

print("Linear Regression: ")
display_scores(lin_reg_scores)

#from sklearn.metrics import mean_squared_error
#housing_predictions = lin_reg.predict(housing_prepared)

# Decision Tree
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train, y_train)
tree_reg_scores = cross_val_score(tree_reg, train, y_train,  
    scoring="r2", cv=10)

print ("Tree Regression: " )
display_scores(tree_reg_scores)

# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(train, y_train)
forest_scores = cross_val_score(forest_reg, train, y_train, 
    scoring="r2", cv=10)

print ("Forest:" )
display_scores(forest_scores)


# Support Vector Machine
#from sklearn.svm import SVR
#svm_linear = SVR(kernel="linear", C=1.0, gamma='auto')
#svm_linear.fit(train, y_train)
#svm_linear_scores = cross_val_score(svm_linear, train, y_train,
#    scoring="r2", cv=10)

#print ("SVM Linear:" )
#display_scores(svm_linear_scores)

# rbf
#svm_rbf = SVR(kernel="rbf", C=1.0, gamma='auto')
#svm_rbf.fit(train, y_train)
#svm_rbf_scores = cross_val_score(svm_rbf, train, y_train, 
#    scoring="r2", cv=10)

#print ("SVM rbf" )
#display_scores(svm_rbf_scores)





Training is done using 350 columns
Linear Regression: 
    Scores: [ 0.32353491  0.5955188   0.45857671  0.57692441  0.62334115  0.5233004
  0.4579369   0.53408989  0.36919079 -0.05846554]
    Mean: 0.440394842084
    Std dev: 0.189750218777
Tree Regression: 
    Scores: [-0.44190576  0.23518675 -0.84575501  0.36318079  0.17690694  0.12878081
  0.16517882  0.28162083  0.43695896  0.38972555]
    Mean: 0.0889878684166
    Std dev: 0.389345727904
Forest:
    Scores: [ 0.46544611  0.50011556  0.22269508  0.54746536  0.61043793  0.48753119
  0.45301922  0.57125111  0.59063383  0.59079683]
    Mean: 0.503939221135
    Std dev: 0.107835181427


'5'